In [1]:
from utils import configuration
from utils.utils import set_seeds
from load_data import load_data

import torch.optim as optim
import torch.nn as nn
import torch

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
)



In [2]:
cfg = 'config/uda.json'
model_cfg = 'config/bert_base.json'
cfg = configuration.params.from_json(cfg)
model_cfg = configuration.model.from_json(model_cfg)
set_seeds(cfg.seed)


In [4]:
data = load_data(cfg)
sup_train_iter = data.sup_data_iter()
# unzsup_train_iter = data.unsup_data_iter()
# sup_test_iter = data.eval_data_iter()
# print([len(loader)  for loader in (sup_train_iter, unzsup_train_iter, sup_test_iter)])


In [5]:
# load model
config = AutoConfig.from_pretrained(model_cfg.model_name_or_path,num_labels=model_cfg.num_labels)
model = AutoModelForSequenceClassification.from_config(config=config)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model = model.to(device)

## for unsup
# criterion = nn.KLDivLoss(reduction='batchmean', log_target=True)
# LSM = nn.LogSoftmax(dim=1)
#optimizer = optim.Adam(model.parameters(), lr=2e-3)
#model = AutoModelForSequenceClassification.from_config(config=config)

In [25]:
a = torch.tensor([1, 2, 3], dtype=float, requires_grad=True)
model.no_grad()
for i, v in enumerate(model.parameters()):
    print(v)
    if i == 1:
        break


AttributeError: 'BertForSequenceClassification' object has no attribute 'no_grad'

In [ ]:
# sup train

# sup_data_iter = itertools.cycle(sup_test_iter)
model.train()
epochs = 100
losses = []
for n in range(epochs):
    for steps, batch in enumerate(sup_train_iter):
        inputs = (t.to(device) for t in batch[:3])
        labels = batch[-1].to(device)
        outputs = model(*inputs)
        loss = criterion(outputs.logits, labels)
        print(f'steps : {n * len(sup_train_iter) + steps + 1} / {epochs * len(sup_train_iter)} loss : {loss}')

        pred = torch.argmax(outputs.logits, dim = 1)
        print(f'pred {pred} label {labels}')
        print(f'result : {torch.sum(pred == labels, dim = 0)}/ {len(labels)}')
        loss.backward()
        losses.append(loss)
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
PATH = "model/train.pt"
torch.save(model.state_dict(), PATH)

In [ ]:
PATH = "model/train.pt"
model.load_state_dict(torch.load(PATH))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(dpi=150)
plt.plot(losses)
plt.title('supervised')
plt.xlabel('steps')
plt.ylabel('loss')
plt.show()
plt.savefig('fig/superviese_losses.png', dpi=3000)



In [ ]:
# del labels
# del inputs
# del outputs
# del loss
# del pred
# torch.cuda.empty_cache()

'''
unsup_data_iter = itertools.cycle(sup_train_iter)
for steps, batch in enumerate(unsup_data_iter):
    ori_outputs = model(*batch[:3])
    aug_outputs = model(*batch[3:])
    loss = criterion(LSM(aug_outputs.logits), LSM(ori_outputs.logits))
    print(f'steps : {steps} loss : {loss}')
    ori_pred = torch.argmax(ori_outputs.logits, dim = 1)
    aug_pred = torch.argmax(aug_outputs.logits, dim = 1)
    print(f'ori_pred {ori_pred} aug_pred {aug_pred}')
    loss.backward()
    optimizer.step()
'''

In [18]:
import torch

device1 = torch.device('cuda')
print(device1)

cuda


In [4]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t,r,a,f)

4294967296 0 0 0


In [13]:
def sharpening_prediction(i : torch.Tensor, temperature:float, log : bool = True):
    i = i / temperature
    if log:
        f = torch.nn.LogSoftmax(dim=1)
    else:
        f = torch.nn.Softmax(dim=1)
    return f(i)

def confidence_based_masking(x: torch.Tensor, beta:float):
    f = torch.nn.LogSoftmax(dim=1)
    y = f(x)
    maxPs, _ = torch.max(y, dim = 1)
    
    pass

In [14]:
import torch

a = torch.rand((10, 2)) * 10
b = sharpening_prediction(a, 0.5)
print(a, b, end = '')
max_b, _ = torch.max(b, dim=1)
uses_batch_id = max_b > 0.5
print(uses_batch_id)

tensor([[8.6023, 9.0657],
        [1.4684, 6.4086],
        [1.8653, 5.9751],
        [8.6026, 5.3899],
        [6.7140, 5.2371],
        [6.6462, 8.0812],
        [1.8522, 3.2141],
        [5.9392, 3.2032],
        [3.4509, 4.8256],
        [8.4353, 3.8566]]) tensor([[-1.2603e+00, -3.3348e-01],
        [-9.8805e+00, -5.1139e-05],
        [-8.2198e+00, -2.6926e-04],
        [-1.6186e-03, -6.4270e+00],
        [-5.0826e-02, -3.0047e+00],
        [-2.9251e+00, -5.5151e-02],
        [-2.7873e+00, -6.3565e-02],
        [-4.1939e-03, -5.4762e+00],
        [-2.8113e+00, -6.2010e-02],
        [-1.0549e-04, -9.1574e+00]])tensor([False, False, False, False, False, False, False, False, False, False])
